## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Richards Bay,ZA,-28.7830,32.0377,74.53,89,93,2.28,overcast clouds
1,1,Chuy,UY,-33.6971,-53.4616,59.11,89,0,5.30,clear sky
2,2,Nikolskoye,RU,59.7035,30.7861,12.00,85,75,13.42,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,78.30,72,21,13.47,moderate rain
4,4,Castro,BR,-24.7911,-50.0119,56.55,72,0,2.57,clear sky


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("Please enter your preference for minimum temperature in Fahrenheit: "))
max_temp = float(input("Please enter your preference for maximum temperature in Fahrenheit: "))

Please enter your preference for minimum temperature in Fahrenheit: 75
Please enter your preference for maximum temperature in Fahrenheit: 90


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,78.30,72,21,13.47,moderate rain
5,5,Mogadishu,SO,2.0371,45.3438,75.29,82,43,15.99,scattered clouds
9,9,Hobart,AU,-42.8794,147.3294,79.00,41,40,17.27,scattered clouds
17,17,Mataura,NZ,-46.1927,168.8643,76.17,47,88,4.94,overcast clouds
18,18,Mahebourg,MU,-20.4081,57.7000,78.80,83,75,10.36,broken clouds
19,19,Saint-Philippe,RE,-21.3585,55.7679,78.80,65,20,8.05,light rain
22,22,Puerto Ayora,EC,-0.7393,-90.3518,75.00,97,36,4.00,moderate rain
25,25,Port Blair,IN,11.6667,92.7500,82.06,67,0,11.52,clear sky
30,30,Hervey Bay,AU,-25.2986,152.8535,88.00,45,40,8.05,scattered clouds
33,33,Carnarvon,AU,-24.8667,113.6333,84.20,100,90,25.32,overcast clouds


In [52]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                172
City                   172
Country                172
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
dtype: int64

In [53]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.30,moderate rain,-23.1203,-134.9692,
5,Mogadishu,SO,75.29,scattered clouds,2.0371,45.3438,
9,Hobart,AU,79.00,scattered clouds,-42.8794,147.3294,
17,Mataura,NZ,76.17,overcast clouds,-46.1927,168.8643,
18,Mahebourg,MU,78.80,broken clouds,-20.4081,57.7000,
19,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,
22,Puerto Ayora,EC,75.00,moderate rain,-0.7393,-90.3518,
25,Port Blair,IN,82.06,clear sky,11.6667,92.7500,
30,Hervey Bay,AU,88.00,scattered clouds,-25.2986,152.8535,
33,Carnarvon,AU,84.20,overcast clouds,-24.8667,113.6333,


In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [61]:
# 7. Drop the rows where there is no Hotel Name.

# Source: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe

# Import dependency
import numpy as np

# Replace empty cells in column with null values.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)

# Drop rows with null values
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

# Confirm results
hotel_df.loc[hotel_df["Hotel Name"] == ""].count()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [62]:
# Confirm DataFrame
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.30,moderate rain,-23.1203,-134.9692,Pension Maro'i
5,Mogadishu,SO,75.29,scattered clouds,2.0371,45.3438,Hotel Juba
9,Hobart,AU,79.00,scattered clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
17,Mataura,NZ,76.17,overcast clouds,-46.1927,168.8643,Ellie's Villa
18,Mahebourg,MU,78.80,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
19,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
22,Puerto Ayora,EC,75.00,moderate rain,-0.7393,-90.3518,Castillo Galapagos
25,Port Blair,IN,82.06,clear sky,11.6667,92.7500,Welcomhotel Bay Island Port Blair
30,Hervey Bay,AU,88.00,scattered clouds,-25.2986,152.8535,Shelly Bay Resort
33,Carnarvon,AU,84.20,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon


In [63]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [67]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))